# wps_convert

#### wps_convert is a process that runs the [convert](https://rvic.readthedocs.io/en/latest/user-guide/conversion/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from rvic.core.config import read_config
from pkg_resources import resource_filename

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("osprey")
print(f"Using osprey on {url}")

Using osprey on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps


In [3]:
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
osprey.convert?

Signature:
osprey.convert(
    uhs_files=None,
    station_file=None,
    domain=None,
    config_file=None,
    loglevel='INFO',
)
Docstring:
A simple conversion utility to provide users with the ability to convert old routing model setups into RVIC parameters.

Parameters
----------
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
uhs_files : ComplexData:mimetype:`text/plain`
    Path to UHS file
station_file : ComplexData:mimetype:`text/plain`
    Path to stations file
domain : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Path to CESM complaint domain file
config_file : ComplexData:mimetype:`text/cfg`
    Path to input configuration file for Convert process

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/code/birds/osprey/</tmp/osprey-venv/lib/python3.8/site-packages/birdy/client/base.py-0>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
# run convert
output = osprey.convert(
    uhs_files = resource_filename("tests", "/data/samples/sample.uh_s2"),
    station_file = resource_filename("tests", "/data/samples/station_file.txt"),
    domain = resource_filename("tests", "/data/samples/sample_routing_domain.nc"),
    config_file = resource_filename("tests", "/data/configs/convert.cfg")
)

Access the output with nc_to_dataset() or auto_construct_outputs() from wps_tools.output_handling

In [6]:
# NBVAL_IGNORE_OUTPUT
from wps_tools.output_handling import nc_to_dataset, auto_construct_outputs

output_data = nc_to_dataset(output.get()[0])
auto_construct_outputs(output.get())

[<class 'netCDF4._netCDF4.Dataset'>
 root group (NETCDF3_CLASSIC data model, file format NETCDF3):
     title: RVIC parameter file
     comment: Output from the RVIC Streamflow Routing Model.
     Conventions: CF-1.6
     history: Created: Wed Jan  6 17:06:51 2021
     source: /tmp/osprey-venv/bin/osprey
     institution: University of Washington
     hostname: pcic-2003
     username: csanders
     references: Based on the initial model of Lohmann, et al., 1996, Tellus, 48(A), 708-721
     version: 1.1.4
     RvicPourPointsFile: station_file.txt
     RvicUHFile: unknown
     RvicFdrFile: unknown
     RvicDomainFile: sample_routing_domain.nc
     featureType: timeSeries
     dimensions(sizes): timesteps(97), outlets(1), nc_chars(256), sources(1), tracers(1)
     variables(dimensions): float64 timesteps(timesteps), int32 full_time_length(), int32 subset_length(), float64 unit_hydrograph_dt(), float64 outlet_lon(outlets), float64 outlet_lat(outlets), int32 outlet_x_ind(outlets), int32 ou

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [7]:
input_config = read_config("tests/data/configs/convert.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicUHFile": "unknown",
    "RvicFdrFile": "unknown",
}

In [8]:
# Metadata of the output of RVIC convert module
metadata = {
    "title": output_data.title,
    "year": output_data.history.split()[5],
    "month": output_data.history.split()[2],
    "day": output_data.history.split()[3].zfill(2),
    "RvicUHFile": output_data.RvicUHFile,
    "RvicFdrFile": output_data.RvicFdrFile,
}
assert metadata == expected